In [24]:
# Description: How to detect and draw contours around objects in 
# an image using OpenCV.

import cv2 # Computer vision library
import svgutils.transform as sg # svg utility library
import sys # System parameter and functions library
import tkinter as tk # TKinter library

root = Tk()
root.geometry("400x100")
root.title("Selection")
Display = Button(root, height = 7,
                 width = 50,
                 text ="Please select an image/file, press to close.",
                 command = root.destroy)
Display.pack()
                 
mainloop()

from tkinter.filedialog import askopenfilename # tkinter GUI used to select file

path = askopenfilename() # show an "Open" dialog box and return the path to the selected file

# Read the color image
image = cv2.imread(path)

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Display the grayscale image
cv2.imshow('Gray image', gray)  
cv2.waitKey(0) # Wait for keypress to continue
cv2.destroyAllWindows() # Close windows

#----------------------------------------
blurred = cv2.GaussianBlur(gray, (3, 3), 0) # gaussian blur
median = cv2.medianBlur(blurred, 1) # median blur
bilateral = cv2.bilateralFilter(median,3,50,50) # bilateral filter
#----------------------------------------
 
# Convert the grayscale image to binary
ret, binary = cv2.threshold(bilateral, 150, 255, # threshold of 150
  cv2.THRESH_OTSU) 
 
# Display the binary image
cv2.imshow('Binary image', binary)
cv2.waitKey(0) # Wait for keypress to continue
cv2.destroyAllWindows() # Close windows
 
# To detect object contours, we want a black background and a white 
# foreground, so we invert the image (i.e. 255 - pixel value)
inverted_binary = ~binary
cv2.imshow('Inverted binary image', inverted_binary)
cv2.waitKey(0) # Wait for keypress to continue
cv2.destroyAllWindows() # Close windows
 
# Find the contours on the inverted binary image, and store them in a list
# Contours are drawn around white blobs.
# hierarchy variable contains info on the relationship between the contours
contours, hierarchy = cv2.findContours(inverted_binary,   #inverted_binary
  cv2.RETR_TREE,
  cv2.CHAIN_APPROX_SIMPLE)

# Find contours, obtain bounding rect, and draw width
##cnts = cv2.findContours(inverted_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
##cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in contours:
    x,y,w,h = cv2.boundingRect(c)
    cv2.putText(image, str(w), (x,y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 1)

cv2.imshow('', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

#Fills in the inside of contours
#cv2.drawContours(image, contours, contourIdx=-1, color=(255,255,255),thickness=-1)
cv2.fillPoly(image,pts=contours,color=(255,255,255))
#Contours list converted to svg
with open("path.svg", "w+") as f:
    f.write(f'<svg width="{w}" height="{h}" xmlns="http://www.w3.org/2000/svg">')

    for c in contours:
        f.write('<path d="M')
        for i in range(len(c)):
            x, y = c[i][0]
            f.write(f"{x} {y} ")
        f.write('" style="stroke:pink"/>')
    f.write("</svg>")


fig = sg.fromfile('path.svg')
fig.set_size(('3508','2480')) #'2480','3508' 
fig.save('scaled.svg')
     
# Draw the contours (in red) on the original image and display the result
# Input color code is in BGR (blue, green, red) format
# -1 means to draw all contours
with_contours = cv2.drawContours(image, contours, -1,(255,0,255),3)
cv2.imshow('Detected contours', with_contours)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Show the total number of contours that were detected
print('Total number of contours detected: ' + str(len(contours)))


#-------------------------------------------------------------------------

from svg_to_gcode.svg_parser import parse_file
from svg_to_gcode.compiler import Compiler, interfaces
gcode_compiler = Compiler(interfaces.Gcode, movement_speed=1000, cutting_speed=300, pass_depth=5)

curves = parse_file("scaled.svg")

gcode_compiler.append_curves(curves)
gcode_compiler.compile_to_file("path.gcode", passes=2)
 
#--------------------------------------------------------------------------    


#-------------------------------------------------------------------------
# Draw just the first contour
# The 0 means to draw the first contour
##first_contour = cv2.drawContours(new_image, contours, 0,(255,0,255),3)
##cv2.imshow('First detected contour', first_contour)
##cv2.waitKey(0)
##cv2.destroyAllWindows()
 
# Draw a bounding box around the first contour
# x is the starting x coordinate of the bounding box
# y is the starting y coordinate of the bounding box
# w is the width of the bounding box
# h is the height of the bounding box
##x, y, w, h = cv2.boundingRect(contours[0])
##cv2.rectangle(first_contour,(x,y), (x+w,y+h), (255,0,0), 5)
##cv2.imshow('First contour with bounding box', first_contour)
##cv2.waitKey(0)
##cv2.destroyAllWindows()
 
# Draw a bounding box around all contours
##for c in contours:
##  x, y, w, h = cv2.boundingRect(c)
 
    # Make sure contour area is large enough
##  if (cv2.contourArea(c)) > 10:
##    cv2.rectangle(with_contours,(x,y), (x+w,y+h), (255,0,0), 5)
         
##cv2.imshow('All contours with bounding box', with_contours)
##cv2.waitKey(0)
##cv2.destroyAllWindows()

#----------------------------------------------------------------


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
